<a href="https://colab.research.google.com/github/c0991100247/2025.07.12-PERIC/blob/main/P_ETHI_02_%E5%8F%B8%E6%B3%95%E9%99%A2%E8%A3%81%E5%88%A4%E6%9B%B8%E4%B8%AD%E7%9A%84%E6%95%91%E8%AD%B7%E6%8A%80%E8%A1%93%E5%93%A1_%E5%A4%9A%E6%A8%A1%E6%85%8B%E5%A4%A7%E5%9E%8B%E8%AA%9E%E8%A8%80%E6%A8%A1%E5%9E%8B%E7%9A%84%E9%81%8B%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai # 用於Gemini API
!pip install -q -U PyMuPDF             # 用於PDF文字擷取 (fitz)

In [ ]:
import os
import json
import re
import fitz # PyMuPDF (for PDF文字擷取)
from tqdm.notebook import tqdm
import google.generativeai as genai
from google.colab import userdata
from datetime import datetime # 用於日期轉換
from google.colab import drive
# drive.mount('/content/drive') # Mount Drive if needed, but keep path general

# --- 步驟 0: 設定組態與載入函式庫 ---

# 重要：請調整這些路徑以符合您儲存文件和希望輸出文件的位置！
# 建議將PDF文件上傳到Colab環境中，或使用Google雲端硬碟並在此處指定正確路徑。
# 範例：如果您將PDF上傳到 Colab 的 /content/pdfs 資料夾
# PDF_FOLDER_PATH = '/content/pdfs'
# 範例：如果您使用 Google 雲端硬碟，且文件在 'My Drive/MyLegalDocs' 資料夾中
# drive.mount('/content/drive') # 如果使用雲端硬碟，請取消註解此行
# PDF_FOLDER_PATH = '/content/drive/MyDrive/MyLegalDocs'

# 請在此處指定您的PDF文件資料夾路徑
PDF_FOLDER_PATH = '/content/drive/MyDrive/' # <-- 請修改此路徑

# 處理後的資料輸出檔案 (將儲存至指定路徑)
# 建議儲存在 Colab 環境中，或 Google 雲端硬碟中的指定位置
# 範例：OUTPUT_JSON_PATH = '/content/processed_documents.json'
# 範例：OUTPUT_JSON_PATH = '/content/drive/MyDrive/processed_legal_documents.json'

# 請在此處指定您的輸出 JSON 檔案路徑
OUTPUT_JSON_PATH = '/content/drive/MyDrive/' # <-- 請修改此路徑

# Gemini API處理文件數量限制 (設定為 None 處理所有文件)
DOCUMENT_PROCESS_LIMIT = None

print("函式庫載入完成，組態路徑已定義。")
print(f"請確認並調整以下路徑以符合您的需求:")
print(f"PDF文件預期路徑: {PDF_FOLDER_PATH}")
print(f"處理後的資料將儲存至: {OUTPUT_JSON_PATH}")

In [ ]:
# --- 步驟 1: 定義PDF文字擷取函式 ---

def extract_text_from_pdf(pdf_path: str) -> str | None:
    """
    使用PyMuPDF從PDF文件中擷取所有文字。
    處理擷取過程中可能發生的錯誤。
    """
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text += page.get_text() # 串接所有頁面的文字
        doc.close()
    except Exception as e:
        print(f"錯誤：從 {pdf_path} 擷取文字失敗: {e}")
        return None
    return text

print("`extract_text_from_pdf` 函式已定義。")



In [ ]:
# --- 步驟 2: 定義案件字號擷取函式 (正規表達式) ---

def extract_case_number(text: str) -> str | None:
    """
    從文件文字中擷取台灣法院的案件字號 (案號)。
    正規化：移除空格，並將 '臺' 替換為 '台'。
    """
    regex_pattern = r'^(?:[\s\S]*?)?(?P<court_name>臺灣[\u4E00-\u9FFF\s]{2,10}?(?:地方法院|高等法院|最高法院))([\u4E00-\u9FFF\s]{1,15})?\s*(?P<year>\d{2,3})年度(?P<case_type>[\u4E00-\u9FFF]{1,5}?)字第(?P<number>\d+)號'
    match = re.search(regex_pattern, text, re.MULTILINE)

    if match:
        court_name = match.group('court_name').strip()
        year = match.group('year')
        case_type = match.group('case_type').strip()
        number = match.group('number')

        full_extracted_id = f"{court_name}{year}年度{case_type}字第{number}號"
        cleaned_id = full_extracted_id.replace(' ', '').replace('臺', '台')
        return cleaned_id
    else:
        return None

print("`extract_case_number` 函式已定義。")



In [ ]:
# --- 步驟 3: 定義主文件處理函式 ---

def process_all_documents(pdf_dir: str, output_file: str) -> list[dict]:
    """
    處理指定資料夾中的所有PDF文件，並將結果儲存為JSON。
    文件ID直接使用PDF檔名 (不含副檔名)。
    """
    processed_documents = {}

    print(f"\n--- 開始處理PDF文件，路徑來自: {pdf_dir} ---")
    if not os.path.exists(pdf_dir):
        print(f"警告: 找不到PDF資料夾，路徑位於 {pdf_dir}。跳過PDF處理。")
        return []

    pdf_files = [f for f in os.listdir(pdf_dir) if f.lower().endswith('.pdf')]
    print(f"找到 {len(pdf_files)} 個PDF文件。")

    for filename in tqdm(pdf_files, desc="處理PDF文件"):
        pdf_path = os.path.join(pdf_dir, filename)
        doc_content = extract_text_from_pdf(pdf_path)

        # 文件ID直接來自PDF檔名 (移除.pdf副檔名)
        document_id = filename[:-4].strip().replace(' ', '').replace('臺', '台')
        document_notes = []

        if doc_content:
            # 嘗試從內容中擷取案件字號 (用於後續Gemini分析參考)
            extracted_case_number_from_content = extract_case_number(doc_content)
            if extracted_case_number_from_content is None:
                document_notes.append("在PDF內容中未能找到標準案件字號。")
            elif extracted_case_number_from_content != document_id:
                document_notes.append(
                    f"文件ID '{document_id}' 與內容擷取到的案件字號 '{extracted_case_number_from_content}' 不符。"
                )

            # 將文件資訊加入處理清單
            processed_documents[document_id] = {
                "document_id": document_id,
                "original_source_format": "PDF",
                "original_filename": filename,
                "full_text": doc_content,
                "extracted_case_number_from_content": extracted_case_number_from_content,
                "notes": document_notes if document_notes else ["文件ID來自檔名。"]
            }
        else:
            print(f"跳過PDF文件 '{filename}'，因文字擷取錯誤 (例如：PDF格式不正確)。")
            document_notes.append("文字擷取失敗。")
            processed_documents[document_id] = {
                "document_id": document_id,
                "original_source_format": "PDF",
                "original_filename": filename,
                "full_text": None,
                "extracted_case_number_from_content": None,
                "notes": document_notes
            }


    print(f"PDF處理完成。目前識別出 {len(processed_documents)} 個文件。")

    final_output_data = list(processed_documents.values())

    # 將整合後的資料儲存至JSON檔案
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(final_output_data, f, ensure_ascii=False, indent=2)
    print(f"所有處理過的文件資料已儲存至Colab環境中的 '{output_file}'。")

    return final_output_data

print("`process_all_documents` 函式已定義。")

# --- 步驟 4: 執行文件處理 ---

# 確保您的Google雲端硬碟已掛載！
# from google.colab import drive
# drive.mount('/content/drive')

final_documents_data = process_all_documents(PDF_FOLDER_PATH, OUTPUT_JSON_PATH)

print(f"\n`process_all_documents` 執行完成。總計識別出獨特文件數: {len(final_documents_data)}")



In [ ]:
# --- 步驟 5: 檢查輸出結果 ---

if os.path.exists(OUTPUT_JSON_PATH):
    with open(OUTPUT_JSON_PATH, 'r', encoding='utf-8') as f:
        processed_data = json.load(f)

    print(f"\n--- 處理後資料範例 ({len(processed_data)} 總文件數) ---")
    for i, doc in enumerate(processed_data[:5]): # 顯示前5個文件
        print(f"\n文件 {i+1}:")
        for key, value in doc.items():
            if key == "full_text" and isinstance(value, str) and len(value) > 200:
                print(f"  {key}: {value[:200]}... (已截斷)")
            else:
                print(f"  {key}: {value}")
    if len(processed_data) > 5:
        print("\n... (僅顯示前5個文件)")

    print(f"\n--- 範例結束 ---")

else:
    print(f"錯誤: 輸出檔案 '{OUTPUT_JSON_PATH}' 未找到。請確認步驟 4 成功執行。")



In [ ]:
# --- 步驟 6: 使用Gemini API分析文件 ---

# 輔助函式：將民國紀年轉換為西元紀年
def convert_roc_to_ad(roc_year_str: str) -> str | None:
    """
    將中華民國紀年轉換為西元紀年。
    例如：輸入 '112' 得到 '2023'。
    """
    try:
        roc_year = int(roc_year_str)
        ad_year = roc_year + 1911
        return str(ad_year)
    except (ValueError, TypeError):
        return None

# 載入已處理的法律文件
if not os.path.exists(OUTPUT_JSON_PATH):
    print(f"錯誤: 輸出檔案 '{OUTPUT_JSON_PATH}' 未找到。請確認先前的步驟已成功執行。")
    # 不退出，讓後續步驟有機會處理其他錯誤
else:
    with open(OUTPUT_JSON_PATH, 'r', encoding='utf-8') as f:
        processed_data_for_gemini = json.load(f)

    # 設定Gemini API
    # 請確保您已在 Colab 環境的 Secrets 中設定 'GOOGLE_API_KEY'
    # 在左側面板找到 '🔑' 圖標，點擊後新增一個密鑰，名稱設定為 'GOOGLE_API_KEY'，值貼上您的 Gemini API 金鑰。
    try:
        genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
    except KeyError:
        print("錯誤: Colab Secrets中未找到 'GOOGLE_API_KEY'。請新增！")
        # 不退出，讓後續步驟有機會處理其他錯誤
    except Exception as e:
        print(f"設定 Gemini API 時發生錯誤: {e}")
        # 不退出，讓後續步驟有機會處理其他錯誤


    # 定義系統指令
    system_instruction_text = """
    您是一位在台灣法律領域具有深厚專業知識的人工智慧助理。
    您專精於分析與《緊急醫療救護法》相關的法律文件，並擁有豐富的法務助理（paralegal）經驗，特別擅長從台灣司法院裁判書系統下載的刑事裁判文書中精準提取關鍵資訊。

    您的核心職責是提供客觀、公正且不帶偏見的分析，旨在清晰描繪緊急醫療救護技術員（EMT）在刑事案件中的專業責任面貌。請務必確保您的資訊提取過程嚴謹，並僅基於文件內容，避免任何推斷或個人意見。

    您的所有回應應遵循指令，並以清晰、結構化的方式呈現。
    """

    # 初始化Gemini模型，優先使用system_instruction參數
    try:
        model = genai.GenerativeModel('gemini-1.5-flash-latest',
                                      system_instruction=system_instruction_text)
        _system_instruction_set_flag = True
    except TypeError:
        print("警告: 模型初始化不直接支援 `system_instruction` 參數。將系統指令附加到使用者提示中。")
        model = genai.GenerativeModel('gemini-1.5-flash-latest')
        _system_instruction_set_flag = False
    except Exception as e:
        print(f"初始化 Gemini 模型時發生錯誤: {e}")
        # 如果模型初始化失敗，後續的 API 呼叫會出錯，但我們還是讓程式繼續，以便使用者看到錯誤訊息

    print("\n--- 開始Gemini API綜合分析 ---")

    updated_processed_data_final = []
    num_processed = 0

    # 根據 DOCUMENT_PROCESS_LIMIT 限制處理的文件數量
    documents_to_process = processed_data_for_gemini
    if DOCUMENT_PROCESS_LIMIT is not None:
        documents_to_process = processed_data_for_gemini[:DOCUMENT_PROCESS_LIMIT]

    # 檢查模型是否成功初始化
    if 'model' not in locals():
         print("錯誤: Gemini 模型未能成功初始化。跳過 API 分析步驟。")
         updated_processed_data_final = processed_data_for_gemini # 將原始資料複製到最終結果
    else:
        for doc in tqdm(documents_to_process, desc="使用Gemini分析文件"):
            full_text = doc.get("full_text", "")
            document_id = doc.get("document_id", "N/A")

            # 跳過內容過短或無明顯法律背景的文件
            if not full_text or len(full_text) < 500:
                doc["extracted_legal_data"] = {"status": "skipped_too_short"}
                if "notes" not in doc:
                    doc["notes"] = []
                elif not isinstance(doc["notes"], list):
                     doc["notes"] = [doc["notes"]]
                doc["notes"].append("跳過綜合分析 (文字過短)。")
                updated_processed_data_final.append(doc)
                continue

            # 針對Gemini的綜合使用者提示
            user_prompt_text = f"""
            請仔細分析以下台灣刑事裁判文件，並精準提取所有要求資訊。請按照指定的 JSON 格式輸出，若某項資訊未找到，請將其值設為 `null` 或空陣列 `[]`（依資料類型而定）。

            文件內容：
            ---
            {full_text[:6000]}
            ---

            請提取以下資訊：

            1.  救護技術員姓名 (emergency_medical_technicians):
                列出文件中所有被提及為「救護技術員」或「救護人員」的個人姓名。姓名通常為繁體中文，且大多數為三個漢字。請將所有找到的姓名收集在一個字串陣列中。

            2.  救護技術員等級 (emt_grade):
                判斷文件中提及的「救護技術員」的等級。請從「初級救護員 (EMT1)」、「中級救護員 (EMT2)」、「高級救護員 (EMTP)」中選擇。請勿將僅有基本CPR或AED訓練的非執業人員（如劉欣華案例）混淆為救護技術員。

            3.  救護技術員雇主 (emt_employer):
                識別文件中提及的救護技術員所受雇的機構或公司。請尋找如「受雇於」或相關語句後的雇主名稱。例如：「救護車有限公司」、「醫院」。

            4.  救護技術員任務 (emt_mission):
                根據文件內容，判斷救護技術員當時執行的是何種勤務或任務。任務可能包括「送病患至醫院」、「返家護送」、「安寧返家」或其他描述。請尋找「執行勤務」等關鍵詞後的任務描述。

            5.  救護技術員法律角色 (emt_role):
                判斷救護技術員在該案件中的法律角色。請從「原告」、「告訴人」、「被告」、「證人」或「被害人」中選擇一個最符合的。每個救護技術員都應有一個角色。

            6.  辯護人存在 (defense_attorney):
                判斷在文件開頭部分是否存在「辯護人」（例如「選任辯護人」）。請以布林值 (true/false) 回應。

            7.  衛福部醫事審議委員會鑑定書存在 (mohw_involve):
                判斷文件中是否提及「衛福部醫事審議委員會鑑定書」或其別名（如「衛生福利部審議」、「委員會鑑定書」）。請以布林值 (true/false) 回應。請勿與交通事故相關資料混淆。

            8.  是否為公訴案件 (public_prosec):
                判斷本案是否為「公訴」案件。請在文件開頭部分尋找「公訴人」，並進一步確認其後是否跟隨「檢察官」。若符合，請以布林值 (true/false) 回應。

            9.  事件發生日期 (date_incident):
                提取事件發生的日期。此資訊常在「經查」或「唯查」段落。請將中華民國紀年轉換為西元紀年，格式為YYYY-MM-DD。

            10. 上訴日期 (date_appeal):
                提取上訴提出的日期。此資訊常在文件上半部（前5段），如「委任***律師向本院聲請」後的日期。若無上訴日期，請設為null。請將中華民國紀年轉換為西元紀年，格式為YYYY-MM-DD。

            11. 裁判日期 (date_trial):
                提取裁判文書的判決日期。此資訊通常在文件末尾，法官姓名列表之後。請勿與書記官後的日期混淆。文件中必定存在此日期。請將中華民國紀年轉換為西元紀年，格式為YYYY-MM-DD。

            12. 事件發生縣市 (county_incident):
                提取事件發生的縣市名稱。此資訊常在「經查」或「唯查」段落。例如：「嘉義縣」、「高雄市」。

            13. 案件字號 (case_id):
                提取文件開頭「裁判字號」或標題後方的案件字號。例如：「臺灣彰化地方法院107年度交易字第810號」。請省略「刑事判決」或「刑事裁定」字樣，並移除空格。請勿與「公訴人檢察官」混淆。

            14. 承辦法院 (case_court):
                從案件字號中提取負責審理本案的法院名稱。例如：「臺灣彰化地方法院」。

            15. 案由 (cause_of_action):
                提取原告上訴（或檢察官起訴）被告的「案由」。此資訊在「裁判案由」或「犯罪事實」段落中至關重要。若有「聲請交付審判」，請繼續解析直至找到真正的案由。例如：「業務過失致重傷害」、「業務過失致死」、「過失傷害」。


            請以單一 JSON 物件回應，包含以下鍵值對：
            {{
              "emergency_medical_technicians": [], // 字串陣列
              "emt_grade": null, // 字串: "初級救護員 (EMT1)", "中級救護員 (EMT2)", "高級救護員 (EMTP)", 或 null
              "emt_employer": null, // 字串
              "emt_mission": null, // 字串
              "emt_role": null, // 字串: "原告", "告訴人", "被告", "證人", "被害人", 或 null
              "defense_attorney": null, // 布林值: true/false
              "mohw_involve": null, // 布林值: true/false
              "public_prosec": null, // 布林值: true/false
              "date_incident": null, // 字串:YYYY-MM-DD 或 null
              "date_appeal": null, // 字串:YYYY-MM-DD 或 null
              "date_trial": null, // 字串:YYYY-MM-DD
              "county_incident": null, // 字串
              "case_id": null, // 字串
              "case_court": null, // 字串
              "cause_of_action": null // 字串
            }}
            """

            final_prompt_for_gemini = user_prompt_text
            if not _system_instruction_set_flag:
                final_prompt_for_gemini = system_instruction_text + "\n" + user_prompt_text

            try:
                response = model.generate_content(
                    final_prompt_for_gemini,
                    generation_config=genai.types.GenerationConfig(
                        temperature=0.0,
                        max_output_tokens=1500
                    ),
                    safety_settings={
                        'HARM_CATEGORY_HARASSMENT':'BLOCK_NONE',
                        'HARM_CATEGORY_HATE_SPEECH':'BLOCK_NONE',
                        'HARM_CATEGORY_SEXUALLY_EXPLICIT':'BLOCK_NONE',
                        'HARM_CATEGORY_DANGEROUS_CONTENT':'BLOCK_NONE'
                    }
                )

                response_text = response.text.strip()
                if response_text.startswith("```json"):
                    response_text = response_text[len("```json"):].strip()
                if response_text.endswith("```"):
                    response_text = response_text[:-len("```")].strip()

                extracted_data = {}
                try:
                    parsed_response = json.loads(response_text)
                    extracted_data["emergency_medical_technicians"] = parsed_response.get("emergency_medical_technicians", [])
                    extracted_data["emt_grade"] = parsed_response.get("emt_grade")
                    extracted_data["emt_employer"] = parsed_response.get("emt_employer")
                    extracted_data["emt_mission"] = parsed_response.get("emt_mission")
                    extracted_data["emt_role"] = parsed_response.get("emt_role")
                    extracted_data["defense_attorney"] = parsed_response.get("defense_attorney")
                    extracted_data["mohw_involve"] = parsed_response.get("mohw_involve")
                    extracted_data["public_prosec"] = parsed_response.get("public_prosec")
                    extracted_data["date_incident"] = parsed_response.get("date_incident")
                    extracted_data["date_appeal"] = parsed_response.get("date_appeal")
                    extracted_data["date_trial"] = parsed_response.get("date_trial")
                    extracted_data["county_incident"] = parsed_response.get("county_incident")
                    extracted_data["case_id"] = parsed_response.get("case_id")
                    extracted_data["case_court"] = parsed_response.get("case_court")
                    extracted_data["cause_of_action"] = parsed_response.get("cause_of_action")

                except json.JSONDecodeError:
                    print(f"警告: Gemini未回傳有效JSON，文件ID: {document_id}。回應: {response_text[:200]}...")
                    extracted_data = {"status": "json_parse_error", "raw_response": response_text}
                except AttributeError as ae:
                     print(f"警告: Gemini回傳非預期結構，文件ID: {document_id}。錯誤: {ae}。回應: {response_text[:200]}...")
                     extracted_data = {"status": "structure_error", "raw_response": response_text}

                doc["extracted_legal_data"] = extracted_data

            except Exception as e:
                print(f"使用Gemini API處理文件 {document_id} 時發生錯誤: {e}")
                doc["extracted_legal_data"] = {"status": "api_error", "error_message": str(e)}
                if "notes" not in doc:
                    doc["notes"] = []
                elif not isinstance(doc["notes"], list):
                     doc["notes"] = [doc["notes"]]
                doc["notes"].append(f"Gemini API綜合分析錯誤: {e}")

            updated_processed_data_final.append(doc)
            num_processed += 1

        print(f"--- Gemini API綜合分析完成。已處理 {num_processed} 個文件。 ---")

        # 將包含 Gemini 分析結果的資料儲存回 JSON 檔案
        try:
            with open(OUTPUT_JSON_PATH, 'w', encoding='utf-8') as f:
                json.dump(updated_processed_data_final, f, ensure_ascii=False, indent=2)
            print(f"包含所有擷取法律欄位的處理後資料已儲存回 '{OUTPUT_JSON_PATH}'。")
        except Exception as e:
            print(f"儲存更新後的 JSON 檔案時發生錯誤: {e}")


    print("\n--- 包含法律欄位的更新後處理資料範例 (前3個經Gemini處理或跳過的文件) ---")
    display_count = 0
    for doc in updated_processed_data_final:
        if "extracted_legal_data" in doc:
            print(f"\n文件 (ID: {doc.get('document_id', 'N/A')}):")
            print(f"  擷取資料: {json.dumps(doc.get('extracted_legal_data', {}), ensure_ascii=False, indent=2)}")
            print(f"  備註: {doc.get('notes', 'N/A')}")
            display_count += 1
        # 也顯示跳過的文件，以確認所有文件都被考慮到
        elif "notes" in doc and "跳過綜合分析" in "".join(doc["notes"]):
             print(f"\n文件 (ID: {doc.get('document_id', 'N/A')}):")
             print(f"  狀態: 已跳過")
             print(f"  備註: {doc.get('notes', 'N/A')}")
             display_count += 1

        if display_count >= 3:
            break
    if display_count == 0:
        print("沒有經Gemini處理或明確標記跳過的文件可供顯示。")

    print("\n--- 範例結束 ---")

In [ ]:
# -*- coding: utf-8 -*-

# --- 步驟 0: 環境準備與設定 ---

# 確保您的Google雲端硬碟已掛載 (如果您的 JSON 檔案儲存在雲端硬碟)
# from google.colab import drive
# drive.mount('/content/drive') # 如果您的 JSON 檔案儲存在雲端硬碟，請取消註解此行

# 安裝處理Excel檔案所需的函式庫 (如果尚未安裝)
!pip install -q openpyxl pandas

import pandas as pd
import json
import os

print("函式庫載入完成，並已掛載Google雲端硬碟 (若需要)。")

# --- 設定檔案路徑 ---
# 請確認這裡的 JSON_FILE_PATH 與您產生 JSON 檔案的路徑一致
# 範例：JSON_FILE_PATH = '/content/processed_documents.json'
# 範例：JSON_FILE_PATH = '/content/drive/MyDrive/processed_legal_documents.json'

# 請在此處指定您的輸入 JSON 檔案路徑
JSON_FILE_PATH = '/content/drive/MyDrive/processed_legal_documents.json' # <-- 請修改此路徑

# 輸出 CSV/Excel 的資料夾 (將根據 JSON 檔案的路徑自動設置)
OUTPUT_DIR = os.path.dirname(JSON_FILE_PATH)
# 如果 JSON_FILE_PATH 是相對路徑或沒有目錄，OUTPUT_DIR 將是空字串或 '.'
# 在這種情況下，我們將輸出檔案儲存在 Colab 的 /content 目錄下
if not OUTPUT_DIR or OUTPUT_DIR == '.':
    OUTPUT_DIR = '/content'

# CSV 輸出檔案路徑
OUTPUT_CSV_FILE = os.path.join(OUTPUT_DIR, 'processed_legal_documents_exploded.csv')

# Excel 輸出檔案路徑
OUTPUT_EXCEL_FILE = os.path.join(OUTPUT_DIR, 'processed_legal_documents_exploded.xlsx')

print(f"將從 '{JSON_FILE_PATH}' 讀取資料。")
print(f"CSV 檔案將儲存至: '{OUTPUT_CSV_FILE}'")
print(f"Excel 檔案將儲存至: '{OUTPUT_EXCEL_FILE}'")

# --- 步驟 1: 讀取 JSON 並轉換為 Pandas DataFrame ---
try:
    with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 將主要資料轉換為 DataFrame
    df = pd.DataFrame(data)

    # 處理 'extracted_legal_data' 巢狀字典
    # 先將 'extracted_legal_data' 轉換為 DataFrame
    # 使用 errors='ignore' 處理可能不存在 'extracted_legal_data' 的行
    extracted_df = pd.json_normalize(df['extracted_legal_data'], errors='ignore')

    # --- 重要修改: 使用 explode() 來展開 'emergency_medical_technicians' 為獨立的列 ---
    if 'emergency_medical_technicians' in extracted_df.columns:
        # 確保 'emergency_medical_technicians' 欄位是列表型態，以便 explode 正常運作
        # 將非列表的值（例如 null, 字符串）轉換為包含單個值的列表或空列表
        extracted_df['emergency_medical_technicians'] = extracted_df['emergency_medical_technicians'].apply(
            lambda x: x if isinstance(x, list) else ([x] if x is not None else [])
        )
        # 執行 explode，讓每個姓名成為一個獨立的列，並複製其他相關欄位
        # 使用 ignore_index=True 以便後續合併
        extracted_df_exploded = extracted_df.explode('emergency_medical_technicians', ignore_index=True)

        # 可選: 移除展開後可能產生的空字串行 (如果原列表中有空字串或 None)
        # extracted_df_exploded = extracted_df_exploded[extracted_df_exploded['emergency_medical_technicians'].astype(bool)]

        # 為了合併，我們需要將原始 df 的行與 exploded 後的行對應起來
        # 最簡單的方法是將原始 df 的每一行與其對應的 exploded 行合併
        # 但 explode 改變了行數，所以直接 join 會複雜
        # 更好的方法是只處理原始 df 中非列表欄位，然後與 exploded 後的 extracted_df 合併
        # 我們需要一個共同的鍵。原始 df 的索引可以作為這個鍵
        df['original_index'] = df.index # 為原始 df 添加索引列

        # 將處理後的 extracted_df_exploded (帶有原始索引資訊) 與原始 df 合併
        # 注意：這一步需要 extracted_df_exploded 中有某種方式可以追蹤回原始 df 的索引
        # explode(ignore_index=True) 會丟失原始索引。
        # 讓我們重新考慮 explode 的使用方式，保留原始索引
        extracted_df_with_original_index = pd.json_normalize(data, record_path='extracted_legal_data', meta=['document_id', 'original_source_format', 'original_filename', 'notes'], errors='ignore')

        # 現在 extracted_df_with_original_index 包含了原始 document_id 和其他元數據
        # 我們可以在這個 DataFrame 上進行 explode
        if 'emergency_medical_technicians' in extracted_df_with_original_index.columns:
             extracted_df_with_original_index['emergency_medical_technicians'] = extracted_df_with_original_index['emergency_medical_technicians'].apply(
                lambda x: x if isinstance(x, list) else ([x] if x is not None else [])
            )
             final_df = extracted_df_with_original_index.explode('emergency_medical_technicians', ignore_index=True)

        # 處理原始 df 中的列表欄位 (例如 notes) - 現在這些已經在 extracted_df_with_original_index 中了
        # 確保 notes 欄位是字串
        if 'notes' in final_df.columns:
            final_df['notes'] = final_df['notes'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)

        # 可選: 處理 'full_text' 欄位
        # 預設：移除 'full_text' 欄位
        if 'full_text' in final_df.columns: # 確保 'full_text' 列存在
             final_df = final_df.drop(columns=['full_text'])

    else:
        # 如果沒有 emergency_medical_technicians 欄位，或者根本沒有 extracted_legal_data
        # 直接將原始 df 扁平化，不進行 explode
        final_df = pd.json_normalize(data, errors='ignore')
        # 處理原始 df 中的列表欄位 (例如 notes)
        for col in ['notes', 'judges']: # 如果您保留了法官姓名擷取，'judges' 會存在
            if col in final_df.columns:
                final_df[col] = final_df[col].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        # 可選: 處理 'full_text' 欄位
        if 'full_text' in final_df.columns:
            final_df = final_df.drop(columns=['full_text'])


    # 重新排列欄位順序，將 extracted_legal_data 相關的欄位放在一起
    # 首先獲取所有欄位名稱
    all_cols = final_df.columns.tolist()

    # 定義您想要放在前面的欄位順序
    preferred_order_prefix = [
        'document_id',
        'original_filename',
        'original_source_format',
        'notes',
        'extracted_legal_data.status', # 如果存在的話
        'extracted_legal_data.error_message', # 如果存在的話
        'extracted_legal_data.raw_response', # 如果存在的話
        'extracted_legal_data.case_id',
        'extracted_legal_data.case_court',
        'extracted_legal_data.cause_of_action',
        'extracted_legal_data.date_trial',
        'extracted_legal_data.date_incident',
        'extracted_legal_data.county_incident',
        'emergency_medical_technicians', # 展開後的 EMT 姓名
        'extracted_legal_data.emt_grade',
        'extracted_legal_data.emt_employer',
        'extracted_legal_data.emt_mission',
        'extracted_legal_data.emt_role',
        'extracted_legal_data.defense_attorney',
        'extracted_legal_data.mohw_involve',
        'extracted_legal_data.public_prosec',
        'extracted_legal_data.date_appeal',
    ]

    # 過濾出實際存在於 final_df 中的 preferred 欄位
    preferred_cols_existing = [col for col in preferred_order_prefix if col in all_cols]

    # 找出所有不在 preferred 列表中的欄位
    other_cols = [col for col in all_cols if col not in preferred_cols_existing]

    # 組合最終的欄位順序 (preferred 欄位在前，其餘欄位在後)
    final_col_order = preferred_cols_existing + other_cols

    # 重新索引 DataFrame 以應用新的欄位順序
    final_df = final_df[final_col_order]


    print("JSON 資料已成功讀取、展開並轉換為 DataFrame。")
    print("DataFrame 前 5 行預覽:")
    display(final_df.head()) # 使用 display 獲得更好的格式
    print(f"DataFrame 形狀: {final_df.shape}")

except FileNotFoundError:
    print(f"錯誤: 未找到 JSON 檔案。請確認路徑 '{JSON_FILE_PATH}' 正確。")
except json.JSONDecodeError:
    print(f"錯誤: JSON 檔案 '{JSON_FILE_PATH}' 格式不正確。")
except Exception as e:
    print(f"處理資料時發生未知錯誤: {e}")

# --- 步驟 2: 儲存為 CSV 和 Excel 檔案 ---
# 只有在 final_df 成功創建時才嘗試儲存
if 'final_df' in locals() and not final_df.empty:
    try:
        final_df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig') # 'utf-8-sig' 避免中文亂碼
        print(f"資料已成功儲存為 CSV 檔案: '{OUTPUT_CSV_FILE}'")
    except Exception as e:
        print(f"儲存 CSV 檔案時發生錯誤: {e}")

    try:
        final_df.to_excel(OUTPUT_EXCEL_FILE, index=False, engine='openpyxl')
        print(f"資料已成功儲存為 Excel 檔案: '{OUTPUT_EXCEL_FILE}'")
    except Exception as e:
        print(f"儲存 Excel 檔案時發生錯誤: {e}")

    print("\n轉換完成！您現在可以在 Google 雲端硬碟或 Colab 環境中找到 CSV 和 Excel 檔案。")
elif 'final_df' in locals() and final_df.empty:
    print("\nDataFrame 為空，未執行 CSV/Excel 儲存。請檢查輸入 JSON 檔案是否包含資料。")
else:
    print("\nDataFrame 未成功建立，未執行 CSV/Excel 儲存。請檢查之前的錯誤訊息。")